In [1]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
img_height = 224
img_width = 224
batch_size = 32
IMAGE_SIZE = (224,224)

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)



In [5]:
training_set = train_datagen.flow_from_directory("/Users/RishiSingh/Computer Science University /Third Year - 2020-2021/Artificial Intelligence/CNN Dataset/AI_N0834113(2)/Logos/Train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory("/Users/RishiSingh/Computer Science University /Third Year - 2020-2021/Artificial Intelligence/CNN Dataset/AI_N0834113(2)/Logos/Test",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 874 images belonging to 29 classes.
Found 219 images belonging to 29 classes.


In [6]:
print(training_set.class_indices)

{'Adidas': 0, 'Apple': 1, 'BMW': 2, 'Citroen': 3, 'Cocacola': 4, 'DHL': 5, 'Fedex': 6, 'Ferrari': 7, 'Ford': 8, 'Google': 9, 'HP': 10, 'Heineken': 11, 'Intel': 12, 'McDonalds': 13, 'Mini': 14, 'Multiple': 15, 'Nbc': 16, 'Nike': 17, 'None': 18, 'Pepsi': 19, 'Porsche': 20, 'Puma': 21, 'Redbull': 22, 'Sprite': 23, 'Starbucks': 24, 'Texaco': 25, 'Unicef': 26, 'Vodafone': 27, 'Yahoo': 28}


In [38]:
vgg = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape= (224,224,3)
)

In [39]:
for layer in vgg.layers:
  layer.trainable = False

In [40]:
Folders = glob('LogosNew/Train/*')

In [41]:
x = Flatten()(vgg.output)
prediction = Dense(len(Folders), activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [43]:
history = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
28/28 [==============================] - 378s 14s/step - loss: 0.1578 - accuracy: 0.9714 - val_loss: 2.2913 - val_accuracy: 0.5845
Epoch 2/5
28/28 [==============================] - 398s 14s/step - loss: 0.1008 - accuracy: 0.9840 - val_loss: 2.3471 - val_accuracy: 0.5388
Epoch 3/5
28/28 [==============================] - 446s 16s/step - loss: 0.0557 - accuracy: 0.9954 - val_loss: 2.2394 - val_accuracy: 0.5434
Epoch 4/5
28/28 [==============================] - 457s 16s/step - loss: 0.0397 - accuracy: 0.9966 - val_loss: 2.1985 - val_accuracy: 0.5571
Epoch 5/5
28/28 [==============================] - 464s 17s/step - loss: 0.0270 - accuracy: 1.0000 - val_loss: 2.1685 - val_accuracy: 0.5525


In [49]:
import keras 
image_path = "/Users/RishiSingh/Computer Science University /Third Year - 2020-2021/Artificial Intelligence/CNN Dataset/flickr_logos_27_dataset 2/Logos/Predictions/image11.jpg"

img = keras.preprocessing.image.load_img(
    image_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
predicted_class_indices=np.argmax(predictions,axis=1)
labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]


print(predictions1)


['BMW']


In [51]:
import os
os.getcwd()

'/Users/RishiSingh/Computer Science University /Third Year - 2020-2021/Artificial Intelligence/CNN Dataset/flickr_logos_27_dataset 2'

In [52]:
model.save(os.path.join(os.getcwd(),"BrandsVGG16.h5"))